In [2]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import os

In [3]:
# reference materials

# https://github.com/zackthoutt/wine-deep-learning/blob/master/scrape-winemag.py
# https://www.kaggle.com/datasets/zynicide/wine-reviews

In [5]:
mavs24_stats_summary_url = 'https://www.basketball-reference.com/teams/DAL/2024.html'

mavs24_stats_summary_page = requests.get(mavs24_stats_summary_url,params={
      'render_js': True})    

mavs24_soup = BeautifulSoup(mavs24_stats_summary_page.text, 'lxml')


In [6]:
teams_list = []

for tag in mavs24_soup.find_all('div', {"class": "division"}):
   if tag.find('a') != None:
      for child_tag in tag.find_all('a'):
         teams_list.append(child_tag.attrs['href'].split('/')[2])

#print(teams_list)

In [10]:
all_team_stats_df = pd.DataFrame()

In [18]:
# may be more efficient to create df once

team_stats_df_names = []

for team in teams_list:
    for year in range(2020,2024):
        team_stats_summary_url = 'https://www.basketball-reference.com/teams/' + team + '/' + str(year) + '.html'
        team_stats_summary_page = requests.get(team_stats_summary_url,params={'render_js': True})
        team_soup = BeautifulSoup(team_stats_summary_page.text, 'lxml')

        team_vs_opp_stats = []
        team_vs_opp_column_names = []
        #games_played = 0

        try:
            for i in range(len(team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' '))):
                if 'data-stat' in team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i]:
                    if "scope" in team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i+1].split('<')[0].strip('>'):
                        pass
                    elif team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i+1].split('<')[0].strip('>') == '':
                        pass
                    elif team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i+1].split('<')[0].strip('>') == 'Year/Year':
                        pass
                    elif team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i].strip('data-stat=') == '"g"':
                        #games_played = team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i+1].split('<')[0].strip('>')
                        pass
                    else:
                        team_vs_opp_stats.append(team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i+1].split('<')[0].strip('>'))
                        team_vs_opp_column_names.append(team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i].strip('data-stat='))
                        #print(team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i].strip('data-stat='), " : ", team_soup.find('div',id = 'all_team_and_opponent').contents[4].split(' ')[i+1].split('<')[0].strip('>'))
        
            team_vs_opp_column_names_unique = [name.strip('"') for name in team_vs_opp_column_names[:23]]
            
            team_vs_opp_df = pd.DataFrame(team_vs_opp_stats)

            reshaped_team_vs_opp_df = pd.DataFrame(np.reshape(team_vs_opp_df.values,(-1,23)))

            reshaped_team_vs_opp_df.columns = team_vs_opp_column_names_unique

            reshaped_team_vs_opp_df['team'] = team
            reshaped_team_vs_opp_df['year'] = year

            #reshaped_team_vs_opp_df.reset_index(inplace = True)
            #reshaped_team_vs_opp_df.drop('index', axis = 1, inplace = True)

            all_team_stats_df = pd.concat([all_team_stats_df,reshaped_team_vs_opp_df])
            
        except:
           print('Error: ' + team + '_' + str(year))

In [21]:
all_team_stats_df

,player,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,fg2,fg2a,...,drb,trb,ast,stl,blk,tov,pf,pts,team,year
0,Team,19955,2992,6718,.445,713,1917,.372,2279,4801,...,2552,3487,1737,577,343,1159,1882,8305,TOR,2014
1,Team/G,243.4,36.5,81.9,.445,8.7,23.4,.372,27.8,58.5,...,31.1,42.5,21.2,7.0,4.2,14.1,23.0,101.3,TOR,2014
2,Lg,2,24,23,22,9,11,9,28,23,...,20,17,21,25,23,9,29,13,TOR,2014
3,Opponent,19955,2957,6573,.450,568,1578,.360,2389,4995,...,2505,3359,1719,596,352,1224,1815,8039,TOR,2014
4,Opponent/G,243.4,36.1,80.2,.450,6.9,19.2,.360,29.1,60.9,...,30.5,41.0,21.0,7.3,4.3,14.9,22.1,98.0,TOR,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933,Team/G,242.1,42.0,87.6,.480,11.0,30.1,.364,31.1,57.5,...,33.1,43.7,25.9,8.3,4.1,14.6,20.5,114.4,NOP,2023
1934,Lg,8,16,17,11,23,29,15,8,10,...,13,13,11,2,24,21,20,15,NOP,2023
1935,Opponent,19855,3357,7105,.472,1002,2953,.339,2355,4152,...,2634,3426,2044,600,386,1228,1673,9223,NOP,2023
1936,Opponent/G,242.1,40.9,86.6,.472,12.2,36.0,.339,28.7,50.6,...,32.1,41.8,24.9,7.3,4.7,15.0,20.4,112.5,NOP,2023


In [22]:
#all_team_stats_df.reset_index(inplace = True)
#all_team_stats_df.drop('index', axis = 1, inplace = True)

all_team_stats_df.to_csv('all_team_stats.csv')

In [4]:
# may need to add error testing - 2024 season monthly game data urls for future months not yet published

months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

visiting_teams = []
home_teams = []
dates = []
visitor_scores = []
home_scores = []

In [18]:
# may need to add error testing - 2024 season monthly game data urls for future months not yet published
# skipping the bubble year for now (2020)

months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
#bubble_months = ['october', 'november', 'december', 'january', 'february', 'march', 'july', 'august']

visiting_teams = []
home_teams = []
dates = []
visitor_scores = []
home_scores = []

for year in range(2023, 2024):
    for month in months:
        game_results_url = 'https://www.basketball-reference.com/leagues/NBA_' + str(year) + '_games-' + month + '.html'
        game_results = requests.get(game_results_url,params={'render_js': True})    
        game_results_soup = BeautifulSoup(game_results.text, 'lxml')
        for tag in game_results_soup.find_all('tr'):
            if tag.find('td') != None:
                visiting_teams.append(tag.find_all('td')[1].attrs['csk'][:3])
                home_teams.append(tag.find_all('td')[1].attrs['csk'][-3:])
                dates.append(tag.find_all('td')[1].attrs['csk'][4:-4])
                try:
                    visitor_scores.append(int(tag.find_all('td')[2].contents[0]))
                except:
                    pass
                try:
                    home_scores.append(int(tag.find_all('td')[4].contents[0]))
                except:
                    pass

game_result_df = pd.DataFrame({'visitor':visiting_teams, 'home':home_teams, 'dates':dates, 'visitor_score':visitor_scores, 'home_score':home_scores})
game_result_df.to_csv('game_results_2023.csv')

In [11]:
game_results_url

'https://www.basketball-reference.com/leagues/NBA_2020_games-april.html'

In [17]:
game_results_soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202311303" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiLo